In [1]:
%config IPCompleter.greedy=True

In [2]:
import csv
import pandas as pd
import dateutil
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import datetime
from scipy.integrate import simps
from numpy import trapz

In [3]:
df = pd.DataFrame.from_csv('twitter_data_new.csv')
df = df.reset_index()

C:\Users\Salman Ahmed\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
'''Convert date string to date object'''
df['date']= pd.to_datetime(df['date'], errors='coerce')

In [5]:
'''cleaning the unknown format string dates'''
idx = df[df['date'].isnull()].index
df = df.drop(df.index[idx])

In [6]:
'''Sorting the data based on date'''
df = df.sort_values('date')

In [7]:
df['retweets'] = pd.to_numeric(df['retweets'], errors='coerce')

In [8]:
idx = df[df['retweets'].isnull()].index
df = df.drop(df.index[idx])

In [9]:
'''Count the total number of tweets and unique twitter users'''
total_count_groups = df.groupby(['project id'])

count_group_data = []
for g, gdf in total_count_groups:
    row_data = []

    #row_data.append(g)
    row_data.append(gdf.iloc[0]['project id'])
    #row_data.append(gdf.iloc[0]['project name'])
    #row_data.append(gdf['date'].min())
    #row_data.append(gdf['date'].max().week)
    row_data.append(gdf['project id'].count() + gdf['retweets'].sum())
    row_data.append(gdf['username'].nunique())
    
    count_group_data.append(row_data)

#print(group_data)
count_group_df = pd.DataFrame(count_group_data)
count_group_df.columns = ['Project Id', 'tweets', 'twitters']
count_group_df.to_csv('project_total_tweets_and_twitters_new.csv', sep=',', encoding='utf-8')

In [10]:
'''Grouping twitter data by project id and date'''
#group_df = df.groupby(['project id', pd.Grouper(key='date', freq='W-MON')])['retweets', 'favorites'].sum().reset_index()
groups = df.groupby(['project id', pd.Grouper(key='date', freq='W-MON')])

group_data = []
for g, gdf in groups:
    row_data = []

    #row_data.append(g)
    row_data.append(gdf.iloc[0]['project id'])
    row_data.append(gdf.iloc[0]['project name'])
    #row_data.append(gdf['date'].min())
    row_data.append(gdf['date'].max().week)
    row_data.append(gdf['date'].max())
    row_data.append(gdf['project id'].count() + gdf['retweets'].sum())
    row_data.append(gdf['username'].nunique())
    
    group_data.append(row_data)

#print(group_data)
group_df = pd.DataFrame(group_data)
group_df.columns = ['project id', 'project name', 'weekno', 'date', 'tweets', 'twitters']

In [11]:
sorted_df = group_df.sort_values(['project id', 'date'])

In [12]:
columns = ['project id', 'project name', 'weekno', 'date', 'tweets', 'twitters', 'days']
new_df = pd.DataFrame(columns=columns).reset_index()
new_df = new_df.fillna(0) # with 0s rather than NaNs

In [13]:
groups = sorted_df.groupby('project id')
frames = []
days = []

for g, gdf in groups:
    days.extend((gdf['date'] - gdf['date'].min()).dt.days.tolist())
    frames.append(gdf)

new_df = pd.concat(frames)
new_df = new_df.assign(days=pd.Series(days).values)

In [14]:
tmp_df = new_df.copy(deep=True)

In [15]:
group_by_ids = new_df.groupby(['project id', 'weekno'])

group_data_new = []
for g, gdf in group_by_ids:
    row_data_new = []

    #row_data.append(g)
    row_data_new.append(gdf.iloc[0]['project id'])
    row_data_new.append(gdf.iloc[0]['project name'])
    #row_data_new.append(gdf['date'].min())
    row_data_new.append(gdf.iloc[0]['weekno'])
    row_data_new.append(gdf['tweets'].sum())
    row_data_new.append(gdf['twitters'].sum())
    row_data_new.append(gdf['days'].sum())
    
    group_data_new.append(row_data_new)

#print(group_data)
group_df_new = pd.DataFrame(group_data_new)
group_df_new.columns = ['project id', 'project name', 'weekno', 'tweets', 'twitters', 'days']

In [16]:
groups = group_df_new.groupby(['project id'])

area_data = []

for g, gdf in groups:
    data = []
    
    x = gdf['weekno'].tolist()
    y = gdf['tweets'].tolist()
    max_tweets = gdf['tweets'].max()
    max_unique_user = gdf['twitters'].max()
    
    xmin = gdf['weekno'].min() - 1
    
    x[:] = [a - xmin for a in x]
    
    area = trapz(y, x)
    
    data.append(g)
    data.append(max_tweets)
    data.append(max_unique_user)
    data.append(area)
    
    area_data.append(data)

area_df = pd.DataFrame(area_data)
area_df.columns = ['Project Id', 'max tweets(weekly)', 'max unique twitters(weekly)', 'area']

In [17]:
'''Calculated the unique project ids to filter out the rest from our main project data source'''
unique_project_ids = area_df['Project Id'].unique()

In [18]:
'''Merging area related data frame and count related data frame'''
merged = count_group_df.merge(area_df, on="Project Id", how="outer").fillna("")

In [19]:
'''Reading the main crowdfunding project data'''
project_data = pd.DataFrame.from_csv('crowdfunding.data.10.17.csv')

'''Filtering some unnecessary features'''
project_data = project_data[['Project Id', 'Name', 'Top Category', 'Category', 'Creator', 'Location', 'Updates', 
             'Comments', 'Rewards', 'Goal', 'Pledged', 'Backers', 'Start', 'End', 'Duration in Days', 
              'Facebook Connected', 'Facebook Friends', 'Facebook Shares', 'Has Video', '# Images', 
             '# Words (Description)', '# Words (Risks and Challenges)']]

'''Cleaning some invalid rows'''
project_data = project_data[np.isfinite(project_data['Duration in Days'])]

C:\Users\Salman Ahmed\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [20]:
'''Dropping some rows from the crowdfunding project data to associate with the area and count data'''
project_df = project_data[project_data['Project Id'].isin(unique_project_ids)]

In [21]:
'''Merging the area data and count data with the crowdfunding project data to create the final dataset'''
final_dataset = project_df.merge(merged, on="Project Id", how="outer").fillna("")

In [22]:
'''Storing the final dataset in a csv formatted file'''
final_dataset.to_csv('final_dataset_new.csv', sep=',', encoding='utf-8')